<a href="https://colab.research.google.com/github/Changanyuexia/BCE/blob/main/bug_report_vadility_classification_with_BERT_and_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Validity dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/drive/MyDrive/Validity dataset/bert_dataset/Thunderbird/train_bert_THUNDERBIRD.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Thunderbird/test_bert_THUNDERBIRD.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Netbeans/train_bert_NETBEANS.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Netbeans/test_bert_NETBEANS.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Mozilla/train_bert_MOZILLA.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Mozilla/test_bert_MOZILLA.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Firefox/test_bert_FIREFOX.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Firefox/train_bert_FIREFOX.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Eclipse/train_bert_ECLIPSE.csv
/content/drive/MyDrive/Validity dataset/bert_dataset/Eclipse/test_bert_ECLIPSE.csv
/content/drive/MyDrive/Validity dataset/cnn_dataset/Thunderbird/train_cnn_THUNDERBIRD.csv
/content/drive/MyDrive/Validity dataset/cnn_dataset/Thu

In [ ]:
import string
path = "/content/drive/MyDrive/Validity dataset"
#dataset_type: cnn, bert
dataset_type = "cnn"
#project: Thunderbird, Netbeans, Mozilla, Firefox, Eclipse
project = "Mozilla"

invalid_file_name = "/content/drive/MyDrive/Validity dataset" + "/" + dataset_type + "_dataset" + "/" + project + "/" + "invalid_" + dataset_type + "_" + project.upper() + ".csv"
valid_file_name = "/content/drive/MyDrive/Validity dataset" + "/" + dataset_type + "_dataset" + "/" + project + "/" + "valid_" + dataset_type + "_" + project.upper() + ".csv"
train_file_name = "/content/drive/MyDrive/Validity dataset" + "/" + dataset_type + "_dataset" + "/" + project + "/" + "train_" + dataset_type + "_" + project.upper() + ".csv"
test_file_name = "/content/drive/MyDrive/Validity dataset" + "/" + dataset_type + "_dataset" + "/" + project + "/" + "test_" + dataset_type + "_" + project.upper() + ".csv"
print(test_file_name)


/content/drive/MyDrive/Validity dataset/cnn_dataset/Mozilla/test_cnn_MOZILLA.csv


In [ ]:
print(train_file_name)
train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

train_df['text'] = train_df['text'].astype(str).tolist()
test_df['text'] = test_df['text'].astype(str).tolist()

train_df['description'] = train_df.apply(lambda row: str(row['title']) + str(row['text']), axis=1)
test_df['description'] = test_df.apply(lambda row: str(row['title']) + str(row['text']), axis=1)


/content/drive/MyDrive/Validity dataset/cnn_dataset/Mozilla/train_cnn_MOZILLA.csv


# CNN

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
# Convert 'description' and 'valid' columns to numpy array
data = pd.concat([pd.read_csv(train_file_name), pd.read_csv(test_file_name)], ignore_index=True)
data["description"] = data["title"].str.cat(data["text"], sep=" ").astype(str)
# 获取连接后的文本和标签
texts = np.array(data["description"])
labels = np.array(data["valid"])

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=fbac93cfebd1cb09ee2fff01b601e6fe703c4315e55d6c65206d36dfa512f4a3
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2023-06-24 09:51:59--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.248.124, 65.8.248.22, 65.8.248.127, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.248.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   146MB/s    in 10s     

2023-06-24 09:52:10 (141 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  
CPU times: user 379 ms, sys: 44.9 ms, total: 424 ms
Wall time: 49.8 s


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.

    Args:
        texts (List[str]): List of text data

    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Truncate or pad sentences to max_len
        if len(tokenized_sent) > max_len:
            tokenized_sent = tokenized_sent[:max_len]
        else:
            tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)

    return np.array(input_ids)

In [ ]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.

    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# # from sentence_transformers import SentenceTransformer
# # Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, 512)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


<ipython-input-11-af6be4a0c69a>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

There are 4604 / 410683 pretrained vectors found.


In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Specify batch_size
    batch_size = 50

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, shuffle = False, stratify = None)
print(val_labels)


# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = \
data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=10)

[1 1 1 ... 1 1 0]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            # pretrained_embedding = torch.from_numpy(pretrained_embedding)
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)

        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [ ]:
import torch.optim as optim

def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)

    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer

In [ ]:
! pip install scikit-learn
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import time
import torch
import numpy as np
from torch import nn
from sklearn.metrics import precision_recall_fscore_support

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def evaluate(model, val_dataloader):
    """Evaluate the model on the validation set."""
    # Put the model into the evaluation mode
    model.eval()

    # Tracking variables
    val_loss = []
    val_accuracy = []
    invalid_precision = []
    invalid_recall = []
    invalid_f1 = []
    valid_precision = []
    valid_recall = []
    valid_f1 = []

    # For each batch in the validation set
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate accuracy
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

        # Calculate precision, recall, and F1-score
        precision, recall, f1, _ = precision_recall_fscore_support(b_labels.cpu(), preds.cpu(), average=None)

        invalid_precision.append(precision[0])
        invalid_recall.append(recall[0])
        invalid_f1.append(f1[0])
        if len(precision) > 1:
            valid_precision.append(precision[1])
            valid_recall.append(recall[1])
            valid_f1.append(f1[1])
        else:
            valid_precision.append(0)
            valid_recall.append(0)
            valid_f1.append(0)

    # Compute the average loss, accuracy, precision, recall, and F1-score over the validation set
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    invalid_precision = np.mean(invalid_precision)
    invalid_recall = np.mean(invalid_recall)
    invalid_f1 = np.mean(invalid_f1)
    valid_precision = np.mean(valid_precision)
    valid_recall = np.mean(valid_recall)
    valid_f1 = np.mean(valid_f1)

    return val_loss, val_accuracy, invalid_precision, invalid_recall, invalid_f1, valid_precision, valid_recall, valid_f1


def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the model."""
    set_seed()

    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Invalid Precision':^17} | {'Invalid Recall':^16} | {'Invalid F1':^12} | {'Valid Precision':^17} | {'Valid Recall':^16} | {'Valid F1':^12} | {'Elapsed':^9}")
    print("-" * 125)

    for epoch_i in range(epochs):
        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy, invalid_precision, invalid_recall, invalid_f1, valid_precision, valid_recall, valid_f1 = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {invalid_precision:^17.2f} | {invalid_recall:^16.2f} | {invalid_f1:^12.2f} | {valid_precision:^17.2f} | {valid_recall:^16.2f} | {valid_f1:^12.2f} | {time_elapsed:^9.2f}")

    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

In [ ]:
# CNN-rand: Word vectors are randomly initialized.
set_seed(42)
cnn_rand, optimizer = initilize_model(vocab_size=len(word2idx),
                                      embed_dim=300,
                                      learning_rate=0.1,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=5)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  | Invalid Precision |  Invalid Recall  |  Invalid F1  |  Valid Precision  |   Valid Recall   |   Valid F1   |  Elapsed 
-----------------------------------------------------------------------------------------------------------------------------
   1    |   0.529994   |  0.498756  |   76.86   |       0.74        |       0.49       |     0.59     |       0.78        |       0.91       |     0.84     |  284.53  
   2    |   0.502226   |  0.492091  |   77.47   |       0.76        |       0.50       |     0.59     |       0.78        |       0.92       |     0.84     |  277.48  
   3    |   0.492361   |  0.493405  |   77.30   |       0.67        |       0.65       |     0.65     |       0.82        |       0.84       |     0.83     |  277.46  
   4    |   0.483687   |  0.478543  |   78.19   |       0.72        |       0.58       |     0.64     |       0.80        |       0.88       |     0.84     |  277.29  
   5    |   0.4

In [ ]:
# CNN-static: fastText pretrained word vectors are used and freezed during training.
set_seed(42)
cnn_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=True,
                                        learning_rate=0.1,
                                        dropout=0.5)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=5)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  | Invalid Precision |  Invalid Recall  |  Invalid F1  |  Valid Precision  |   Valid Recall   |   Valid F1   |  Elapsed 
-----------------------------------------------------------------------------------------------------------------------------
   1    |   0.541288   |  0.509691  |   76.24   |       0.75        |       0.46       |     0.56     |       0.77        |       0.92       |     0.83     |   81.58  
   2    |   0.511946   |  0.504204  |   76.71   |       0.77        |       0.45       |     0.56     |       0.76        |       0.93       |     0.84     |   80.93  
   3    |   0.503461   |  0.501249  |   77.02   |       0.67        |       0.64       |     0.65     |       0.82        |       0.84       |     0.83     |   80.76  
   4    |   0.497801   |  0.487955  |   77.51   |       0.73        |       0.55       |     0.62     |       0.79        |       0.89       |     0.84     |   80.68  
   5    |   0.4

# BERT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
print(torch.__version__)
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler, RandomSampler
from transformers import BertForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import trange
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from transformers import BartModel, BartTokenizer

2.0.1+cu118


In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)


# model_name = "distilbert-base-uncased" #faster
# tokenizer = DistilBertTokenizer.from_pretrained(model_name, do_lower_case=True)


tokenizer

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
# Configuration Settings
NUM_LABELS = 2
BATCH_SIZE = 8
MAX_LEN = 512
EPOCHS = 5
LEARNING_RATE = 1e-5
# LEARNING_RATE = 5e-5

In [ ]:
def process_data(df, tokenizer, max_len=30):
    """
    Process the data to feed into the pretrained model
    """
    data_processed = tokenizer.batch_encode_plus(df.description.values, pad_to_max_length=True, truncation=True, add_special_tokens=True, return_attention_mask=True, max_length=max_len, return_tensors='pt')
    input_ids_train = data_processed["input_ids"]
    attention_mask_train = data_processed["attention_mask"]
    output_train = torch.tensor(df.valid.values)
    return input_ids_train, attention_mask_train, output_train

input_ids_train, attention_mask_train, output_train = process_data(train_df, tokenizer, max_len=MAX_LEN)
input_ids_test, attention_mask_test, output_test = process_data(test_df, tokenizer, max_len=MAX_LEN)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
def load_data(seq, mask, output, batch_size=32, use_train=True):
    """
    Load data into batches
    """
    data = TensorDataset(seq, mask, output)
    if use_train: sampler = RandomSampler(data)
    else: sampler = SequentialSampler(data)
    dl_data = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dl_data

dl_train = load_data(input_ids_train, attention_mask_train, output_train, batch_size=BATCH_SIZE)
dl_test = load_data(input_ids_test, attention_mask_test, output_test, batch_size=BATCH_SIZE, use_train=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #train on gpu if available
print("Using device {}.\n".format(device))
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS, output_hidden_states=False, output_attentions=False)
# model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS, output_hidden_states=False, output_attentions=False)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE)  # Default optimization
# Learning rate scheduling is applied after optimizer’s update to adjust the learning rate based on the number of epochs.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dl_train) * EPOCHS) #number of batches * number of epochs
# print(model)

Using device cuda.



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.we

In [ ]:
def count_parameters(model):
    ''' Count the total number of trainable parameters '''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(model)
print("The model has total {} trainable parameters".format(num_params))

The model has total 66955010 trainable parameters


In [ ]:
def eval_metric(predictions, labels):
  ''' Calculate average accuracy of the data samples '''
  max_predictions = predictions.argmax(axis=1, keepdim=True) #[batch_size]
  avg_acc = round(accuracy_score(y_true=labels.to('cpu').tolist(), y_pred=max_predictions.detach().cpu().numpy()), 2)*100
  return avg_acc

In [ ]:
def train_fn(model, train_loader, optimizer, device, scheduler, criterion=None):
  ''' Define the training function '''
  model.train() #set the model on train mode
  total_loss, total_acc = 0, 0

  for batch in train_loader:
    batch = tuple(item.to(device) for item in batch)
    input_ids, input_mask, labels = batch
    optimizer.zero_grad() # clear gradients from last batch
    outputs = model(input_ids, attention_mask=input_mask, labels=labels) # get predictions
#     loss = criterion(batch_y_pred, batch_y) # compute the loss
    loss = outputs.loss
    total_loss += loss.item() #aggregate the losses
    loss.backward() # compute the gradients
    optimizer.step() # update the parameters
    scheduler.step()
    logits = outputs.logits
    total_acc += eval_metric(logits, labels)

  loss_per_epoch = total_loss/len(train_loader)
  acc_per_epoch = total_acc/len(train_loader)
  return loss_per_epoch, acc_per_epoch

def eval_fn(model, data_loader, device, criterion=None):
  ''' Define the evaluation function '''
  model.eval() # set the model on eval mode
  total_loss, total_acc = 0, 0

  with torch.no_grad(): # do not need to update the parameters
    for batch in data_loader:
      batch = tuple(item.to(device) for item in batch)
      input_ids, input_mask, labels = batch
      outputs = model(input_ids, attention_mask=input_mask, labels=labels) # get predictions
#       loss = criterion(batch_y_pred, batch_y) # compute the loss
      loss = outputs.loss
      total_loss += loss.item() #aggregate the losses
      logits = outputs.logits
      total_acc += eval_metric(logits, labels)

  loss_per_epoch = total_loss/len(data_loader)
  acc_per_epoch = total_acc/len(data_loader)
  return loss_per_epoch, acc_per_epoch

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, roc_auc_score
from tqdm import trange

# Training
train_losses, test_losses = [], [] # Store train and validation losses for plotting
train_accuracies, test_accuracies = [], []
best_test_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epoch"):
  train_loss_per_epoch, train_acc_per_epoch = train_fn(model, dl_train, optimizer, device, scheduler)
  test_loss_per_epoch, test_acc_per_epoch = eval_fn(model, dl_test, device)

  train_losses.append(train_loss_per_epoch)
  test_losses.append(test_loss_per_epoch)
  train_accuracies.append(train_acc_per_epoch)
  test_accuracies.append(test_acc_per_epoch)

  if test_loss_per_epoch < best_test_loss:
    best_test_loss = test_loss_per_epoch
    torch.save(model.state_dict(), '/content/drive/MyDrive/model.pt')

  print("Epoch: {}, Train Loss: {:.4f}, Train Accuracy: {:.2f}%".format(epoch, train_loss_per_epoch, train_acc_per_epoch))
  print("Epoch: {}, test Loss: {:.4f}, test Accuracy: {:.2f}%\n".format(epoch, test_loss_per_epoch, test_acc_per_epoch)) # Test

# Loading the best trained model
print("Loading the best trained model")
model.load_state_dict(torch.load('/content/drive/MyDrive/model.pt'))

# Create a DataLoader for the test data
test_dataset = TensorDataset(input_ids_test, attention_mask_test)
test_loader = DataLoader(test_dataset, batch_size=32)  # Adjust batch_size to fit your GPU

preds = []
model = model.to(device)
model.eval()

# Predict in batches
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask)
        logits = outputs.logits
        batch_preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        preds.extend(batch_preds)

print(classification_report(output_test, preds))
print("ROC AUC Score: {}".format(roc_auc_score(y_true=output_test, y_score=preds)))


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

# Pipeline

In [ ]:
from transformers import pipeline

pipe = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True , device=0)

# LIME

In [ ]:
!pip install lime

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
class_labels = ["invalid", "valid"]
lime_explainer = LimeTextExplainer(class_names=class_labels)

In [ ]:
def predict_proba_fn(texts):
    a=pipe(texts)
    rt=[]
    for i in a:
        prb=[]
        for j in i:
            prb.append(j['score'])
        rt.append(prb)
    return np.array(rt)

In [ ]:
text = """[patch] To add a File->New->Window menuitem and remove doubleclick in folder pane opens a new window. Currently, double clicking on something in the folder pane opens a new window with that folder selected.  To me, this is bad UI, as opening a new window is an expensive operation and not something most people are going to want.  In addition to this, the new window doesnt provide a different UI at all, its just a duplicate,  so basically you double click and you get two copies of the exact same window. This patch removes the opening of a new window on double click, and adds a new menuitem that opens a new window with the current folder selected.  I chose F5 for the accelerator key, but if someone feels something different is better, by all means.  Also, if for some reason the double click to open makes sense in nntp account (I dont have one, so I dont know),  I can easily add a serverType check to restrict the no new-window behaviour to imap and pop3 accounts.
Reproducible: Always.
Steps to Reproduce:  1. Double click a folder.
Actual Results: new window is opened with the same ui as the original window.
Expected Results: nothing
"""

In [ ]:
# Use the LIME explainer to generate an explanation
exp = lime_explainer.explain_instance(text, predict_proba_fn)

# Display the explanation visualization in the notebook, with text included
exp.show_in_notebook(text=True)

# SHAP

In [ ]:
import shap
import scipy as sp

def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)
shap_values = explainer([text])

# plot the explanation
shap.plots.text(shap_values)


# Transformer Intepret

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

cls_explainer = SequenceClassificationExplainer(model, tokenizer)
attributions = cls_explainer(text)

# Iterate through the list of attributions
for word, score in attributions:
    print(f"Word: {word} - Score: {score}")

# Get predicted class
predicted_class_name = cls_explainer.predicted_class_name

print(f"Predicted class name: {predicted_class_name}")

# Visualize explanation for the first class
cls_explainer.visualize("trans_389374.html")